In [1]:
library(dplyr)
library(reshape2)
library(hipathia)
library(tidyr)
library(VennDiagram)
library(purrr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Loading required package: AnnotationHub

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:igraph’:

    normalize, path, union


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, base

In [2]:
proj_tcga_sel <- c("tcga_brca", "tcga_coad","tcga_esca","tcga_lgg","tcga_luad","tcga_lusc","tcga_paad","tcga_skcm","tcga_stad")

In [3]:
proj_tumorvsctrl <- data.frame(tcga=c("tcga_brca", "tcga_coad","tcga_esca","tcga_lgg","tcga_luad","tcga_lusc","tcga_paad","tcga_skcm","tcga_stad"),
                          gtex=c("Breast","Colon","Esophagus","Brain","Lung","Lung","Pancreas","Skin","Stomach"))
rownames(proj_tumorvsctrl) <- proj_tumorvsctrl$tcga

In [4]:
drug_target_sex_funct <- function(project){

    files_kstest <- list.files(paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/05_kstest_pvadj_logfc_kovsctrl/",project,"/",sep=""))
    ks_logfc <- do.call(rbind, lapply(paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/05_kstest_pvadj_logfc_kovsctrl/",project,"/",files_kstest,sep=""), 
      FUN=function(x) readRDS(x)))
    return(ks_logfc)

}
drug_target_sex_tcga <- do.call(rbind,lapply(proj_tcga_sel, FUN=function(x) drug_target_sex_funct(x)))

drug_target_sex_tcga_one_sex <- data.frame(drug_target_sex_tcga %>% filter(kspvadj < 0.05) %>% group_by(circ, project, drug_target) %>%
summarise(n=n()) %>% filter(n==1)) 

drug_target_sex_tcga_one_sex$sex <- unlist(lapply(1:length(drug_target_sex_tcga_one_sex$circ), FUN=function(x)
data.frame(drug_target_sex_tcga %>% filter(kspvadj < 0.05 & 
                                circ == drug_target_sex_tcga_one_sex$circ[x] &
                                drug_target == drug_target_sex_tcga_one_sex$drug_target[x] &
                                project == drug_target_sex_tcga_one_sex$project[x]
                               ))$sex))

drug_target_sex_tcga_both_sex <- data.frame(drug_target_sex_tcga %>% filter(kspvadj < 0.05) %>% group_by(circ, project, drug_target) %>%
summarise(n=n()) %>% filter(n==2)) %>% mutate(sex = case_when(n==2 ~ "both_sex"))                                                  

drug_target_sex_tcga_sex <- rbind(drug_target_sex_tcga_both_sex, drug_target_sex_tcga_one_sex)
drug_target_sex_tcga_sex$n <- NULL       

drug_target_sex_tcga_sex$tis_ctrl <- proj_tumorvsctrl[drug_target_sex_tcga_sex$project,"gtex"]                                                  

`summarise()` has grouped output by 'circ', 'project'. You can override using
the `.groups` argument.
`summarise()` has grouped output by 'circ', 'project'. You can override using
the `.groups` argument.


In [5]:
length(unique(drug_target_sex_tcga_sex[drug_target_sex_tcga_sex$project == "tcga_brca","drug_target"]))

[1] 29

In [6]:
files_kstest_gtex <- list.files("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/05_kstest_pvadj_logfc_kovsctrl/gtex/")

drug_target_sex_gtex <- do.call(rbind, lapply(paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/05_kstest_pvadj_logfc_kovsctrl/gtex/",files_kstest_gtex,sep=""), 
      FUN=function(x) readRDS(x)))

drug_target_sex_gtex_one_sex <- drug_target_sex_gtex %>% filter(tissue %in% unique(proj_tumorvsctrl$gtex) & kspvadj < 0.05) %>%
group_by(ko,circ,tissue) %>% summarise(n=n()) %>% filter(n!=2)

drug_target_sex_gtex_one_sex$sex <- unlist(lapply(1:length(drug_target_sex_gtex_one_sex$circ), FUN=function(x)
data.frame(drug_target_sex_gtex %>% filter(kspvadj < 0.05 &
                                circ == drug_target_sex_gtex_one_sex$circ[x] &
                                ko == drug_target_sex_gtex_one_sex$ko[x] &
                                tissue == drug_target_sex_gtex_one_sex$tissue[x]))$sex))

drug_target_sex_gtex_both_sex <- drug_target_sex_gtex %>% filter(tissue %in% unique(proj_tumorvsctrl$gtex) & kspvadj < 0.05) %>%
group_by(ko,circ,tissue) %>% summarise(n=n()) %>% filter(n==2) %>% mutate(sex = case_when(n==2 ~ "both_sex"))     

drug_target_sex_gtex_sex <- rbind(drug_target_sex_gtex_both_sex, drug_target_sex_gtex_one_sex)
drug_target_sex_gtex_sex$n <- NULL                                                  

`summarise()` has grouped output by 'ko', 'circ'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'ko', 'circ'. You can override using the
`.groups` argument.


In [7]:
drug_target <- merge(drug_target_sex_tcga_sex,drug_target_sex_gtex_sex, by.x=c("circ","drug_target","tis_ctrl"), by.y=c("circ","ko","tissue"),all.x=TRUE)
colnames(drug_target) <- c("circ","drug_target","tissue_ctrl","project","sign_tumor","sign_ctrol")

pathways <- load_pathways("hsa")
drug_target$name_circ <- get_path_names(pathways,drug_target$circ)

Loaded 146 pathways



In [8]:
length(unique(drug_target[drug_target$project == "tcga_brca","drug_target"]))

[1] 29

In [9]:
circs_sig_dif <- readRDS("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/03_limma_pathvals/limma_pathvals.rds")                                        

circs_sig_dif_both_sex <- data.frame(circs_sig_dif %>% filter(adj.P.Val < 0.05) %>% group_by(circ, project) %>%
summarise(n=n()) %>% filter(n==2) %>% mutate(sex = case_when(n==2 ~ "both_sex")))

circs_sig_dif_one_sex <- data.frame(circs_sig_dif %>% filter(adj.P.Val < 0.05) %>% group_by(circ, project) %>%
summarise(n=n()) %>% filter(n==1))

circs_sig_dif_one_sex$sex <- unlist(lapply(1:length(circs_sig_dif_one_sex$circ), FUN=function(x)
data.frame(circs_sig_dif %>% filter(adj.P.Val < 0.05 &
                                circ == circs_sig_dif_one_sex$circ[x] &
                                project == circs_sig_dif_one_sex$project[x]))$sex))

circs_sig_sex <- rbind(circs_sig_dif_one_sex, circs_sig_dif_both_sex)
circs_sig_sex$n <- NULL                                           

`summarise()` has grouped output by 'circ'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'circ'. You can override using the
`.groups` argument.


In [10]:
drug_target_circs_sig <- merge(drug_target,circs_sig_sex,by.x=c("name_circ","project"), 
                               by.y=c("circ","project"),all.x=TRUE)

In [11]:
colnames(drug_target_circs_sig)[colnames(drug_target_circs_sig) == "sex"] <- "circs_sign_tvsc"

In [12]:
drug_target_circs_sig$sign_ctrol[is.na(drug_target_circs_sig$sign_ctrol)] <- "no_sign"

In [13]:
drug_target_circs_sig$path <- unlist(lapply(drug_target_circs_sig$name_circ, FUN=function(x) unlist(strsplit(x,":"))[1]))

In [14]:
data.frame(drug_target_circs_sig %>% filter(sign_ctrol== "no_sign") %>% group_by(path,project,sign_tumor) %>% summarise(n=n())) %>%
group_by(path) %>% summarise(n=n()) %>% filter(n>5)

`summarise()` has grouped output by 'path', 'project'. You can override using
the `.groups` argument.


path,n
<chr>,<int>
Complement and coagulation cascades,7
ErbB signaling pathway,7
Fc gamma R-mediated phagocytosis,7
Gap junction,6
GnRH signaling pathway,6
Insulin signaling pathway,6
MAPK signaling pathway,9
Natural killer cell mediated cytotoxicity,8
Phospholipase D signaling pathway,6


In [15]:
calc_recurrence_function <- function(project){
    
    pat_drug_corr <- readRDS(paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/04_mm_ko_drug_used/04_1_correc_drug/",project,"/pat_drug_corr_",project,".rds",sep=""))
    patients_geneexp_clind_drug <- readRDS(paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/02_selection_samples/02_1_ext_information_patients_tcga/",project,"/patients_geneexp_clind_drug_",project,".rds",sep=""))
    pat_drug_db_circ <- readRDS(paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/04_mm_ko_drug_used/04_2_pat_drug_db_circ/",project,"/pat_drug_db_circ_",project,".rds",sep=""))
    calc_rec <- merge(unique(patients_geneexp_clind_drug[,c("bcr_patient_barcode","gender","has_new_tumor_events_information")]), 
      pat_drug_corr,by.x="bcr_patient_barcode",by.y="bcr_patient_barcode",all.x== TRUE)

    calc_rec_sum <- data.frame(calc_rec %>% group_by(drug_name, gender, has_new_tumor_events_information) %>% summarise(n_pat=n()))
    calc_rec_total <- data.frame(calc_rec %>% group_by(drug_name, gender) %>% summarise(n_total_pat=n()))

    rec <- merge(calc_rec_sum, calc_rec_total, by.x=c("drug_name","gender"), by.y=c("drug_name","gender"), all.x=TRUE)
    rec$tas_rec <- rec$n_pat/rec$n_total_pat*100 

    rec$project <- rep(project, length(rec$tas_rec))

    rec <- data.frame(rec %>% filter(has_new_tumor_events_information == "YES" & drug_name != ""))

    rec_annot <- merge(unique(pat_drug_db_circ[,c("entrez_id","drug_name")]),
      rec, by.x="drug_name", by.y="drug_name")[,c("drug_name","entrez_id","gender","tas_rec","project")]

    summary_rec <- rec_annot %>%
      group_by(drug_name, entrez_id, project, gender) %>%
      pivot_wider(names_from = gender, values_from = tas_rec, names_prefix = "tas_rec_") %>%
      arrange(drug_name, entrez_id, project)
    
    return(summary_rec)
}    

In [16]:
tas_rec_tcga <- do.call(rbind,lapply(proj_tcga_sel, FUN=function(x) calc_recurrence_function(x)))

`summarise()` has grouped output by 'drug_name', 'gender'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'drug_name'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'drug_name', 'gender'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'drug_name'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'drug_name', 'gender'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'drug_name'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'drug_name', 'gender'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'drug_name'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'drug_name', 'gender'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'drug_name'. You can override using the
`.groups` 

In [17]:
summary_results <- merge(drug_target_circs_sig, tas_rec_tcga, by.x=c("project","drug_target"), by.y=c("project","entrez_id"),all.x=TRUE)

In [18]:
dim(summary_results)

[1] 1895   12

In [19]:
length(unique(summary_results[summary_results$project == "tcga_brca","drug_target"]))

[1] 29

In [20]:
summary_results$circs_sign_tvsc[is.na(summary_results$circs_sign_tvsc)] <- "no_sign"

In [21]:
summary_results$project <- gsub("_","-",toupper(summary_results$project))

In [22]:
summary_results$circ <- NULL

In [23]:
ex_bad_drug_adm <- data.frame(summary_results %>% filter(circs_sign_tvsc == "no_sign" &  sign_ctrol != "no_sign" & sign_tumor != "both_sex" & (tas_rec_FEMALE == 100 | tas_rec_MALE == 100)))

In [24]:
dim(ex_bad_drug_adm)

[1] 14 11

In [25]:
ex_good_drug_adm <- summary_results %>% filter(circs_sign_tvsc == "both_sex" &  sign_ctrol == "no_sign" & sign_tumor == "both_sex" & (tas_rec_FEMALE == 100 | tas_rec_MALE == 100))

In [26]:
dim(ex_good_drug_adm)

summary_results

[1] 64 11

project,drug_target,name_circ,tissue_ctrl,sign_tumor,sign_ctrol,circs_sign_tvsc,path,drug_name,tas_rec_FEMALE,tas_rec_MALE
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
TCGA-BRCA,1956,FoxO signaling pathway: FBXO32,Breast,both_sex,both_sex,no_sign,FoxO signaling pathway,lapatinib,100,NA
TCGA-BRCA,1956,FoxO signaling pathway: PCK1,Breast,both_sex,both_sex,no_sign,FoxO signaling pathway,lapatinib,100,NA
TCGA-BRCA,1956,FoxO signaling pathway: BCL2L11,Breast,both_sex,both_sex,no_sign,FoxO signaling pathway,lapatinib,100,NA
TCGA-BRCA,1956,ErbB signaling pathway: ABL1,Breast,both_sex,both_sex,male,ErbB signaling pathway,lapatinib,100,NA
TCGA-BRCA,1956,FoxO signaling pathway: GADD45G*,Breast,both_sex,both_sex,no_sign,FoxO signaling pathway,lapatinib,100,NA
TCGA-BRCA,1956,Gap junction: GJA1 GJA1,Breast,FEMALE,no_sign,both_sex,Gap junction,lapatinib,100,NA
TCGA-BRCA,1956,ErbB signaling pathway: CDKN1B,Breast,FEMALE,no_sign,both_sex,ErbB signaling pathway,lapatinib,100,NA
TCGA-BRCA,1956,ErbB signaling pathway: STAT5A*,Breast,both_sex,both_sex,both_sex,ErbB signaling pathway,lapatinib,100,NA
TCGA-BRCA,1956,FoxO signaling pathway: CAT,Breast,both_sex,both_sex,no_sign,FoxO signaling pathway,lapatinib,100,NA


In [27]:
write.csv(summary_results, file="/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/06_drug_target_comparation_sex/drug_target_comp_recurrence.csv", row.names=F)  
write.csv(ex_bad_drug_adm, file="/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/06_drug_target_comparation_sex/examp_bad_drug_adm.csv", row.names=F)  
write.csv(ex_good_drug_adm, file="/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/06_drug_target_comparation_sex/examp_good_drug_adm.csv", row.names=F)  

In [28]:
tcga_name <- 
data.frame("name"=
c("Breast invasive carcinoma (TCGA-BRCA)",
"Colon adenocarcinoma (TCGA-COAD)",
"Esophageal carcinoma (TCGA-ESCA)",
"Brain Lower Grade Glioma (TCGA-LGG)",
"Lung adenocarcinoma (TCGA-LUAD)",
"Lung squamous cell carcinoma (TCGA-LUSC)",
"Pancreatic adenocarcinoma (TCGA-PAAD)",
"Skin Cutaneous Melanoma (TCGA-SKCM)",
"Stomach adenocarcinoma (TCGA-STAD)"))
rownames(tcga_name) <- proj_tcga_sel

In [53]:
venn_diagram_funct <- function(project){
    
    # Get unique drug targets for each group
    ko_sign_female_tcga <- unique(drug_target_sex_tcga[drug_target_sex_tcga$kspvadj < 0.05 & drug_target_sex_tcga$project == project & drug_target_sex_tcga$sex == "FEMALE", "drug_target"])
    ko_sign_male_tcga <- unique(drug_target_sex_tcga[drug_target_sex_tcga$kspvadj < 0.05 & drug_target_sex_tcga$project == project & drug_target_sex_tcga$sex == "MALE", "drug_target"])

    pat_drug_db_circ <- readRDS(paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/04_mm_ko_drug_used/04_2_pat_drug_db_circ/", project, "/pat_drug_db_circ_", project, ".rds", sep = ""))

    ko_sign_female_gtex <- unique(drug_target_sex_gtex[drug_target_sex_gtex$ko %in% unique(pat_drug_db_circ$entrez_id) & drug_target_sex_gtex$kspvadj < 0.05 & 
        drug_target_sex_gtex$tissue == proj_tumorvsctrl[project, "gtex"] & drug_target_sex_gtex$sex == "female", "ko"])

    ko_sign_male_gtex <- unique(drug_target_sex_gtex[drug_target_sex_gtex$ko %in% unique(pat_drug_db_circ$entrez_id) & drug_target_sex_gtex$kspvadj < 0.05 & 
        drug_target_sex_gtex$tissue == proj_tumorvsctrl[project, "gtex"] & drug_target_sex_gtex$sex == "male", "ko"])
    
    # Define the AAAS color palette
    aaas_colors <- c("#D16103", "#844288", "#008280", "#B2B2B2")  # AAAS palette colors
    
    venn.plot <- venn.diagram(
        x = list(
            Set1 = ko_sign_female_tcga,
            Set2 = ko_sign_female_gtex,
            Set3 = ko_sign_male_tcga,
            Set4 = ko_sign_male_gtex
        ),
        category.names = c("TUMOR FEMALE", "CTROL FEMALE", "TUMOR MALE", "CTROL MALE"),
        filename = paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/06_drug_target_comparation_sex/venn_diagram_", project, ".png", sep = ""),
        output = TRUE, 
        imagetype = "png",
        height = 500,
        width = 500,
        resolution = 100,
        compression = "lzw",
        fill = aaas_colors,  # Apply AAAS colors here
        linewidth = 1,  
        cat.dist = 0.1,  
        cat.pos = 1,        
        scaled = FALSE, 
        cex = 1.5,
        cat.cex = 1.2  # Adjust size of the category labels
    )
}


In [54]:
lapply(proj_tcga_sel, FUN=function(x) venn_diagram_funct(x))

[[1]]
[1] 1

[[2]]
[1] 1

[[3]]
[1] 1

[[4]]
[1] 1

[[5]]
[1] 1

[[6]]
[1] 1

[[7]]
[1] 1

[[8]]
[1] 1

[[9]]
[1] 1

In [31]:
drug_effec_funct <- function(project){

    ko_sign_female_tcga <- unique(drug_target_sex_tcga[drug_target_sex_tcga$kspvadj < 0.05 & drug_target_sex_tcga$project == project & drug_target_sex_tcga$sex=="FEMALE","drug_target"])
    ko_sign_male_tcga <- unique(drug_target_sex_tcga[drug_target_sex_tcga$kspvadj < 0.05 & drug_target_sex_tcga$project == project & drug_target_sex_tcga$sex=="MALE","drug_target"])

    pat_drug_db_circ <- readRDS(paste("/mnt/lustre/scratch/CBRA/research/projects/tcga_mm/results/04_mm_ko_drug_used/04_2_pat_drug_db_circ/",project,"/pat_drug_db_circ_",project,".rds",sep=""))

    ko_sign_female_gtex <- unique(drug_target_sex_gtex[drug_target_sex_gtex$ko %in% unique(pat_drug_db_circ$entrez_id) & drug_target_sex_gtex$kspvadj < 0.05 & 
    drug_target_sex_gtex$tissue ==  proj_tumorvsctrl[project,"gtex"] & drug_target_sex_gtex$sex == "female","ko"])

    ko_sign_male_gtex <- unique(drug_target_sex_gtex[drug_target_sex_gtex$ko %in% unique(pat_drug_db_circ$entrez_id) & drug_target_sex_gtex$kspvadj < 0.05 & 
    drug_target_sex_gtex$tissue ==  proj_tumorvsctrl[project,"gtex"] & drug_target_sex_gtex$sex == "male","ko"])

    drug_ef_tumor_female <- unique(data.frame(pat_drug_db_circ %>% filter(pat_drug_db_circ$entrez_id %in% setdiff(ko_sign_female_tcga,ko_sign_female_gtex)))[,c("entrez_id","drug_name")])
    drug_ef_tumor_female$project <- rep(paste(gsub("_","-",toupper(project)), "female", sep=" "), length(drug_ef_tumor_female$drug_name))
    drug_ef_tumor_male <- unique(data.frame(pat_drug_db_circ %>% filter(pat_drug_db_circ$entrez_id %in% setdiff(ko_sign_male_tcga,ko_sign_male_gtex)))[,c("entrez_id","drug_name")])
    drug_ef_tumor_male$project <- rep(paste(gsub("_","-",toupper(project)), "male", sep=" "), length(drug_ef_tumor_male$drug_name))

    if (nrow(drug_ef_tumor_male) != 0 | nrow(drug_ef_tumor_female) != 0){
        drug_ef_tumor <- rbind(drug_ef_tumor_female, drug_ef_tumor_male)
        drug_ef_tumor$rownames <- paste(drug_ef_tumor$drug_name, " (",drug_ef_tumor$entrez_id,")",sep="")

        drug_ef_tumor <- dcast(drug_ef_tumor, project ~ rownames)

        rownames(drug_ef_tumor) <- drug_ef_tumor$project
        drug_ef_tumor[!is.na(drug_ef_tumor)] <- "YES"
        drug_ef_tumor[is.na(drug_ef_tumor)] <- "NO"
        drug_ef_tumor$project <- rownames(drug_ef_tumor)

        return(drug_ef_tumor)
    }
}

In [91]:
my_list <- lapply(proj_tcga_sel, FUN=function(x) drug_effec_funct(x))

Using rownames as value column: use value.var to override.

Using rownames as value column: use value.var to override.

Using rownames as value column: use value.var to override.

Using rownames as value column: use value.var to override.

Using rownames as value column: use value.var to override.



In [92]:
my_list

[[1]]
                          project clodronic acid (292) denosumab (8600)
TCGA-BRCA female TCGA-BRCA female                  YES              YES
TCGA-BRCA male     TCGA-BRCA male                   NO               NO
                 palonosetron (3359) tamoxifen (2100) tamoxifen (5599)
TCGA-BRCA female                 YES              YES              YES
TCGA-BRCA male                   YES               NO              YES

[[2]]
                          project cetuximab (2215) regorafenib (1969)
TCGA-COAD female TCGA-COAD female              YES                 NO
TCGA-COAD male     TCGA-COAD male              YES                YES
                 regorafenib (2263)
TCGA-COAD female                YES
TCGA-COAD male                  YES

[[3]]
NULL

[[4]]
                        project bevacizumab (713) imatinib (5156)
TCGA-LGG female TCGA-LGG female               YES             YES
TCGA-LGG male     TCGA-LGG male                NO             YES
                tamoxifen (2100)
TCGA-LGG female              YES
TCGA-LGG male                 NO

[[5]]
NULL

[[6]]
NULL

[[7]]
                          project doxorubicin (8644)
TCGA-PAAD female TCGA-PAAD female                YES
TCGA-PAAD male     TCGA-PAAD male                YES

[[8]]
NULL

[[9]]
                          project cetuximab (2215) epirubicin (5321)
TCGA-STAD female TCGA-STAD female              YES               YES
TCGA-STAD male     TCGA-STAD male              YES               YES

In [33]:
df_list

# Get all unique TCGA values
all_tcga <- unique(unlist(lapply(df_list, function(df) df$project)))

# Get all unique drug columns
all_drugs <- unique(unlist(lapply(df_list, function(df) colnames(df)[-1])))

ERROR: Error in eval(expr, envir, enclos): object 'df_list' not found


In [90]:
df_list

ERROR: Error in eval(expr, envir, enclos): object 'df_list' not found


In [ ]:
df_list <- df_list[!sapply(df_list, is.null)]
projects <- lapply(1:length(df_list), FUN=function(x) unlist(rownames(df_list[[x]])))
drugs <- lapply(1:length(df_list), FUN=function(x) unlist(colnames(df_list[[x]])))                 

In [ ]:
df_list